# Linear Classification

 Implement Linear Classification using pytorch. This consists of having fully connected layers connected one after the other and ReLu activation functions between them.
 
 Build a neural network with a minimun of 2 layers in order to do classification.

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
import numpy as np
import torch.utils.data as utils
import time
import pdb
from torch.utils.data.sampler import SubsetRandomSampler
%matplotlib inline

torch.manual_seed(1)    # reproducible

In [2]:
def get_train_valid_loader(data_dir='../data',
                           batch_size=64,
                           augment=False,
                           random_seed = 1,
                           valid_size=0.02,
                           shuffle=True,
                           show_sample=False,
                           num_workers=0,
                           pin_memory=False):
    """
    Utility function for loading and returning train and valid
    multi-process iterators over the CIFAR-10 dataset. A sample
    9x9 grid of the images can be optionally displayed.
    If using CUDA, num_workers should be set to 1 and pin_memory to True.
    Params
    ------
    - data_dir: path directory to the dataset.
    - batch_size: how many samples per batch to load.
    - augment: whether to apply the data augmentation scheme
      mentioned in the paper. Only applied on the train split.
    - random_seed: fix seed for reproducibility.
    - valid_size: percentage split of the training set used for
      the validation set. Should be a float in the range [0, 1].
    - shuffle: whether to shuffle the train/validation indices.
    - show_sample: plot 9x9 sample grid of the dataset.
    - num_workers: number of subprocesses to use when loading the dataset.
    - pin_memory: whether to copy tensors into CUDA pinned memory. Set it to
      True if using GPU.
    Returns
    -------
    - train_loader: training set iterator.
    - valid_loader: validation set iterator.
    """
    error_msg = "[!] valid_size should be in the range [0, 1]."
    assert ((valid_size >= 0) and (valid_size <= 1)), error_msg

    normalize = transforms.Normalize(
        mean=[0.4914, 0.4822, 0.4465],
        std=[0.2023, 0.1994, 0.2010],
    )

    # define transforms
    valid_transform = transforms.Compose([
            transforms.ToTensor(),
            normalize,
    ])
    if augment:
        train_transform = transforms.Compose([
            transforms.RandomCrop(32, padding=4),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            normalize,
        ])
    else:
        train_transform = transforms.Compose([
            transforms.ToTensor(),
            normalize,
        ])

    # load the dataset
    train_dataset = datasets.CIFAR10(
        root=data_dir, train=True,
        download=True, transform=train_transform,
    )

    valid_dataset = datasets.CIFAR10(
        root=data_dir, train=True,
        download=True, transform=valid_transform,
    )

    num_train = len(train_dataset)
    indices = list(range(num_train))
    split = int(np.floor(valid_size * num_train))

    if shuffle:
        np.random.seed(random_seed)
        np.random.shuffle(indices)

    train_idx, valid_idx = indices[split:], indices[:split]
    train_sampler = SubsetRandomSampler(train_idx)
    valid_sampler = SubsetRandomSampler(valid_idx)

    train_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=batch_size, sampler=train_sampler,
        num_workers=num_workers, pin_memory=pin_memory,
    )
    valid_loader = torch.utils.data.DataLoader(
        valid_dataset, batch_size=batch_size, sampler=valid_sampler,
        num_workers=num_workers, pin_memory=pin_memory,
    )

    # visualize some images
    if show_sample:
        sample_loader = torch.utils.data.DataLoader(
            train_dataset, batch_size=9, shuffle=shuffle,
            num_workers=num_workers, pin_memory=pin_memory,
        )
        data_iter = iter(sample_loader)
        images, labels = data_iter.next()
        X = images.numpy().transpose([0, 2, 3, 1])
        plot_images(X, labels)

    return (train_loader, valid_loader)

trainloader, valloader = get_train_valid_loader()

Files already downloaded and verified
Files already downloaded and verified


# Neural Net
We use a very simple convolutional neural net, with two convolutions, each followed by a RELU. After that we have a couple of linear filters which ends up with the 10 classes.  
The reason for this setup was to experiment with convolutions, as well to use the pre-existing setup for the lab. 

In [17]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        ################################################################################
        #                                OUR CODE                                      #
        ################################################################################
        self.kernel_size = 3
        
        self.conv1 = nn.Conv2d(3, 6, self.kernel_size)
        self.conv2 = nn.Conv2d(6, 16, self.kernel_size)
        self.maxpool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(16 * 6 * 6, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        ################################################################################
        #                              END OF OUT CODE                                 #
        ################################################################################


    def forward(self, x):
        ################################################################################
        #                                OUR CODE                                      #
        ################################################################################
        '''
        Forward pass of neural net. Input-size = (3, 32, 32) Output = (10,)
        '''
        
        x = F.relu(self.conv1(x))
        x = self.maxpool(x)
        x = F.relu(self.conv2(x))
        x = self.maxpool(x)
        x = x.view(-1, 16 * 6 * 6)
        
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        
        ################################################################################
        #                              END OF OUR CODE                                 #
        ################################################################################
        return x



In [18]:
################################################################################
#                                OUR CODE                                      #
################################################################################

net = Net()     # define the network
print(net)  # net architecture

# Loss and Optimizer (Try different learning rates)
# Softmax is internally computed.
# Set parameters to be updated. 

optimizer = torch.optim.Adam(net.parameters(), lr = 0.002) # Choose the optimizer you want and tune its hyperparameter
criterion = torch.nn.CrossEntropyLoss()  # the target label is NOT an one-hotted

################################################################################
#                              END OF OUR CODE                                 #
################################################################################

Net(
  (conv1): Conv2d(3, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (maxpool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [19]:
#traindataset = utils.TensorDataset(X_train, y_train)
#trainloader = utils.DataLoader(traindataset, batch_size=64, shuffle=True)

epochs = 5
steps = 0
print_every = 50

for e in range(epochs):
    start = time.time()
    print('\n Epoch nr.{}'.format(e))
   
    # Reduce learning rate on third epoch as we did not see performance gain after epoch 3
    if e == 2:
        optimizer = torch.optim.Adam(net.parameters(), lr = 0.0005) # Choose the optimizer you want and tune its hyperparameter
    
    for images, labels in iter(trainloader):
        steps += 1
        ################################################################################
        #                                OUR CODE                                      #
        ################################################################################
        
        
        optimizer.zero_grad()
        
        output = net(images)
        
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        
        ################################################################################
        #                              END OF OUR CODE                                 #
        ################################################################################
    
        if steps % print_every == 0:
            stop = time.time()
            # Test accuracy
            net.eval()
            correct = 0
            total = 0
            with torch.no_grad():
                for data in valloader:
                    images, labels = data
                    outputs = net(images)
                    _, predicted = torch.max(F.softmax(outputs).data, 1)
                    total += labels.size(0)
                    correct += (predicted == labels).sum().item()

                print('Accuracy of the network on the %d val images step %d: \
                %d %%' % (total, steps, 100 * correct / total))

            start = time.time()


 Epoch nr.0


/Users/theo/anaconda3/envs/deep/lib/python3.7/site-packages/ipykernel_launcher.py:45: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Accuracy of the network on the 1000 val images step 50:                 29 %
Accuracy of the network on the 1000 val images step 100:                 31 %
Accuracy of the network on the 1000 val images step 150:                 36 %
Accuracy of the network on the 1000 val images step 200:                 37 %
Accuracy of the network on the 1000 val images step 250:                 43 %
Accuracy of the network on the 1000 val images step 300:                 41 %
Accuracy of the network on the 1000 val images step 350:                 44 %
Accuracy of the network on the 1000 val images step 400:                 44 %
Accuracy of the network on the 1000 val images step 450:                 45 %
Accuracy of the network on the 1000 val images step 500:                 46 %
Accuracy of the network on the 1000 val images step 550:                 47 %
Accuracy of the network on the 1000 val images step 600:                 49 %
Accuracy of the network on the 1000 val images step 650:         

After training, the model should be saved to be tested on the test dataset or to be used in a real-life application. To save a model in pytorch:

In [20]:
torch.save(net.state_dict(), 'model.ckpt')

To load a pretrained model:

In [21]:
checkpoint = torch.load("./model.ckpt")
net.load_state_dict(checkpoint)

<All keys matched successfully>

In [23]:
def run(X):
    '''
    Test function for run.py file
    '''
    net = Net()
    checkpoint = torch.load("./model.ckpt")
    net.load_state_dict(checkpoint)
    pred = np.empty(0)
    l = np.empty(0)
    net.eval()
    with torch.no_grad():
        for data in valloader:
            images, labels = data
            outputs = net(images)
            _, predicted = torch.max(F.softmax(outputs).data, 1)
            pred = np.append(pred,predicted.numpy())
            l = np.append(l,labels.numpy())

    return pred.astype(int), l.astype(int)
pred, labels = run(valloader)

    
    

/Users/theo/anaconda3/envs/deep/lib/python3.7/site-packages/ipykernel_launcher.py:15: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  from ipykernel import kernelapp as app


In [25]:
# Test run function
np.mean(pred == labels)

0.619